In [1]:
# Data
DATABASE = 'gscholar'
USER = 'admin'
PASSWORD = 'cgbio12#$'
ENDPOINT = 'cgbio-keyword.ci2chv7x8rmn.us-east-2.rds.amazonaws.com'
PORT = '3306'

In [2]:
!pip install mysql
!pip install mysql-connector-python

     |████████████████████████████████| 88 kB 2.8 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.0.3-cp37-cp37m-linux_x86_64.whl size=100150 sha256=ccac2cd9bc6e1d0e80f39a77333a7a55237014f7c8b7c97cf1a5fb78bdfe68e9
  Stored in directory: /root/.cache/pip/wheels/79/1c/f8/11fafab45fe6696eea63794a5d747b9c6b54990ac6f1885fb7
Successfully built mysqlclient
     |████████████████████████████████| 30.9 MB 74 kB/s 


In [3]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd

In [ ]:
# supposing we have the data

In [ ]:
# We use the generated proxies via proxy scraper
# Upload or create beforehand the necessary files
protocols = ['http', 'socks4', 'socks5']

for p, i in zip(protocols, range(len(protocols))):
  filename = p+'.txt'
  proxies_raw = pd.read_csv(filename,  sep="::", names=["Address:port", "Country", "Region", "City"])
  proxies_raw.insert(0, "Protocol", p)
  if i == 0: proxies = proxies_raw
  else: proxies = pd.concat([proxies, proxies_raw])

proxies

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  import sys


,Protocol,Address:port,Country,Region,City
0,http,1.119.181.30:3128,China,Beijing,Beijing
1,http,1.186.34.45:80,India,Tamil Nadu,Chennai
2,http,3.213.139.74:8888,United States,Virginia,Ashburn
3,http,5.16.0.97:1256,Russia,St.-Petersburg,St Petersburg
4,http,5.58.33.187:55507,Ukraine,Ternopil Oblast,Ternopil
...,...,...,...,...,...
84,socks5,202.95.11.216:1080,Singapore,None,None
85,socks5,212.227.30.76:9000,Germany,Rheinland-Pfalz,Katzenelnbogen
86,socks5,217.147.175.20:1088,Ukraine,None,None
87,socks5,221.141.87.130:1080,South Korea,Ulsan,Ulsan


In [ ]:
DATABASE = 'gscholar'
engine = create_engine("mariadb://{}:{}@{}:{}/{}?charset=utf8".format(USER, PASSWORD, ENDPOINT, PORT, DATABASE), encoding='utf8')

with engine.connect() as conn:
  result = conn.execute(text("SHOW DATABASES;"))
  # result = conn.execute(text("SHOW DATABASES;"))

  print(result.all())

[('gscholar',), ('information_schema',), ('innodb',), ('mysql',), ('performance_schema',), ('proxies',)]


In [ ]:
DATABASE = 'proxies'
engine = create_engine("mariadb://{}:{}@{}:{}/{}?charset=utf8".format(USER, PASSWORD, ENDPOINT, PORT, DATABASE), encoding='utf8')

# Preprocessing to avoid problems
proxies.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)

# Upload
proxies.to_sql(con=engine, name='proxies', if_exists='replace',chunksize=100, index=False) 

In [ ]:
read_data = pd.read_sql('proxies', engine )
read_data

,Protocol,Address:port,Country,Region,City
0,http,1.119.181.30:3128,China,Beijing,Beijing
1,http,1.186.34.45:80,India,Tamil Nadu,Chennai
2,http,3.213.139.74:8888,United States,Virginia,Ashburn
3,http,5.16.0.97:1256,Russia,St.-Petersburg,St Petersburg
4,http,5.58.33.187:55507,Ukraine,Ternopil Oblast,Ternopil
...,...,...,...,...,...
1943,socks5,202.95.11.216:1080,Singapore,None,None
1944,socks5,212.227.30.76:9000,Germany,Rheinland-Pfalz,Katzenelnbogen
1945,socks5,217.147.175.20:1088,Ukraine,None,None
1946,socks5,221.141.87.130:1080,South Korea,Ulsan,Ulsan


In [ ]:
DATABASE = 'gscholar'
engine = create_engine("mariadb://{}:{}@{}:{}/{}?charset=utf8".format(USER, PASSWORD, ENDPOINT, PORT, DATABASE), encoding='utf8')

with engine.connect() as conn:
  result = conn.execute(text("SHOW DATABASES;"))
  # result = conn.execute(text("SHOW DATABASES;"))

  print(result.all())

[('gscholar',), ('information_schema',), ('innodb',), ('mysql',), ('performance_schema',), ('proxies',)]


In [ ]:
DATABASE = 'gscholar'
engine = create_engine("mariadb://{}:{}@{}:{}/{}?charset=utf8".format(USER, PASSWORD, ENDPOINT, PORT, DATABASE), encoding='utf8')

with engine.connect() as conn:
  result = conn.execute(text("USE gscholar;"))
  # result = conn.execute(text("SHOW DATABASES;"))

read_data = pd.read_sql('gscholar', engine )
read_data

,Protocol,Address:port,Country,Region,City,Success,Blocked,Timeout
0,socks4,154.79.244.38:10801,Kenya,None,None,36,0,0
1,socks4,91.237.235.53:3629,Ukraine,Odessa,Odessa,2,0,0
2,http,43.129.232.191:6969,Hong Kong,Central and Western District,Central,0,0,0
3,socks4,91.243.164.24:5678,Iran,None,None,0,0,0
4,http,77.236.252.187:1256,Russia,Moscow,Moscow,0,0,0
...,...,...,...,...,...,...,...,...
1855,socks4,185.126.47.75:5678,Lebanon,None,None,0,0,1
1856,http,185.255.47.109:8080,Iraq,None,None,0,0,1
1857,socks4,116.212.142.188:5678,Cambodia,Phnom Penh,Phnom Penh,1,0,1
1858,socks4,102.176.160.70:5678,Guinea,Conakry Region,Conakry,29,1,0


In [ ]:
with engine.connect() as conn:
  result = conn.execute(text("USE gscholar;"))
  # result = conn.execute(text("SHOW DATABASES;"))

with engine.connect() as conn:
  result = conn.execute(text("SHOW TABLES;"))
  # result = conn.execute(text("SHOW DATABASES;"))

  print(result.all())

[]


In [ ]:
read_data = pd.read_sql('data', engine )
read_data

,Protocol,Address:port,Country,Region,City,Success,Blocked,Timeout
0,socks4,45.184.200.233:4153,Brazil,Amazonas,Manaus,7,0,0
1,socks4,124.41.228.175:5678,Nepal,None,None,1,0,0
2,http,129.226.189.147:6969,Hong Kong,Central and Western District,Central,1,0,0
3,http,123.31.43.63:8888,Vietnam,None,None,0,0,0
4,socks4,139.60.12.6:1088,United States,Utah,Ogden,0,0,0
...,...,...,...,...,...,...,...,...
20455,socks4,124.41.228.175:5678,Nepal,None,None,39,1,0
20456,http,129.226.189.147:6969,Hong Kong,Central and Western District,Central,13,0,1
20457,http,123.31.43.63:8888,Vietnam,None,None,1,1,0
20458,socks4,124.109.22.174:5678,Indonesia,None,None,0,0,1
